In [2]:
import numpy as np
import matplotlib.pyplot as plt
import os
import pathlib

import tensorflow as tf


print(tf.__version__)

train_data_dir = pathlib.Path("./data/train")
val_data_dir = pathlib.Path("./data/validation")
test_data_dir = pathlib.Path("./data/test")


2.3.0


In [3]:
train_image_count = len(list(train_data_dir.glob('*/*.jpg')))
test_image_count = len(list(test_data_dir.glob('*/*.jpg')))
val_image_count = len(list(val_data_dir.glob('*/*.jpg')))

train_list_ds = tf.data.Dataset.list_files(str(train_data_dir/'*/*'))
val_list_ds = tf.data.Dataset.list_files(str(val_data_dir/'*/*'))
test_list_ds = tf.data.Dataset.list_files(str(test_data_dir/'*/*'))

In [24]:
class_names = np.array([item.name for item in train_data_dir.glob('*')])
print(f"Class names: {class_names}")

#data augmentation function
IMG_WIDTH=100
IMG_HEIGHT=100

def decode_img(img):
  img = tf.image.decode_jpeg(img, channels=3)
  img = tf.image.convert_image_dtype(img, tf.float32)
  return tf.image.resize(img, [IMG_WIDTH, IMG_HEIGHT])

def get_label(path):
  parts = tf.strings.split(path, os.path.sep)
  return parts[-2] == class_names[0]

def process_path(file_path):
  img = tf.io.read_file(file_path)
  img = decode_img(img)
  label = get_label(file_path)
  return img, label

Class names: ['swift' 'wagonr']


In [90]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

train_labeled_ds = train_list_ds.map(process_path, num_parallel_calls=AUTOTUNE)
#train_labeled_ds = train_labeled_ds.cache().shuffle()
#train_labeled_ds = train_labeled_ds.repeat()
train_ds = train_labeled_ds.cache().shuffle(buffer_size=1000).repeat().batch(25).prefetch(buffer_size=AUTOTUNE)
#train_labeled_ds = train_labeled_ds.prefetch(buffer_size=AUTOTUNE)

test_labeled_ds = test_list_ds.map(process_path, num_parallel_calls=AUTOTUNE)
#test_labeled_ds = test_labeled_ds.cache()
test_ds = test_labeled_ds.cache().batch(25).prefetch(buffer_size=AUTOTUNE)
#test_labeled_ds = test_labeled_ds.prefetch(buffer_size=AUTOTUNE)

val_labeled_ds = val_list_ds.map(process_path, num_parallel_calls=AUTOTUNE)
val_ds = val_labeled_ds.cache().batch(25).prefetch(buffer_size=AUTOTUNE)

In [76]:
for image, label in train_labeled_ds.take(10):
  print("Image shape: ", image.numpy().shape)
  print("Label: ", label.numpy())

Image shape:  (100, 100, 3)
Label:  True
Image shape:  (100, 100, 3)
Label:  True
Image shape:  (100, 100, 3)
Label:  False
Image shape:  (100, 100, 3)
Label:  False
Image shape:  (100, 100, 3)
Label:  False
Image shape:  (100, 100, 3)
Label:  True
Image shape:  (100, 100, 3)
Label:  True
Image shape:  (100, 100, 3)
Label:  False
Image shape:  (100, 100, 3)
Label:  True
Image shape:  (100, 100, 3)
Label:  True


In [92]:
print(model.summary())



Model: "sequential_19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_19 (Flatten)         (None, 30000)             0         
_________________________________________________________________
dense_38 (Dense)             (None, 64)                1920064   
_________________________________________________________________
dense_39 (Dense)             (None, 1)                 65        
Total params: 1,920,129
Trainable params: 1,920,129
Non-trainable params: 0
_________________________________________________________________
None


In [91]:
batch_size = 32
img_height = 100
img_width = 100
steps_per_epoch = np.ceil(train_image_count/batch_size)
input_shape = (100, 100, 3)
num_outputs = 1
num_epochs = 10


import tensorflow as tf
from tensorflow import keras

model = keras.Sequential([
        keras.layers.Flatten(input_shape=input_shape),
        keras.layers.Dense(64, activation='relu'),
        keras.layers.Dense(1)
    ])

model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

tensorboard_callback = keras.callbacks.TensorBoard()

training_history = model.fit(
    train_ds, 
    validation_data=val_ds, 
    epochs=10,
    steps_per_epoch=steps_per_epoch,
    callbacks=[tensorboard_callback])

print(f"Average test loss: {np.average(training_history.history['loss'])}")
val_loss, val_acc = model.evaluate(val_ds, verbose=2)
test_loss, test_acc = model.evaluate(test_ds, verbose=2)

print(f"\nTest accuracy: {test_acc}")
print(f"\nValidation accuracy: {val_acc}")


Epoch 1/10
75/75 [==============================] - 4s 47ms/step - loss: 1.6598 - accuracy: 0.5237 - val_loss: 0.6912 - val_accuracy: 0.5350
Epoch 2/10
75/75 [==============================] - 1s 19ms/step - loss: 0.6987 - accuracy: 0.5685 - val_loss: 0.6882 - val_accuracy: 0.6112
Epoch 3/10
75/75 [==============================] - 1s 20ms/step - loss: 0.7224 - accuracy: 0.5883 - val_loss: 0.6713 - val_accuracy: 0.6313
Epoch 4/10
75/75 [==============================] - 1s 19ms/step - loss: 0.7189 - accuracy: 0.5851 - val_loss: 0.9958 - val_accuracy: 0.5250
Epoch 5/10
75/75 [==============================] - 1s 19ms/step - loss: 0.7068 - accuracy: 0.6165 - val_loss: 0.8695 - val_accuracy: 0.5512
Epoch 6/10
75/75 [==============================] - 1s 19ms/step - loss: 0.6329 - accuracy: 0.6379 - val_loss: 0.6765 - val_accuracy: 0.6200
Epoch 7/10
75/75 [==============================] - 1s 19ms/step - loss: 0.6556 - accuracy: 0.6224 - val_loss: 0.6528 - val_accuracy: 0.6050
Epoch 8/10
75

In [81]:
# Classifiers
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
# Metrics
from sklearn.metrics import precision_recall_fscore_support

In [ ]:
%load_ext tensorboard

%tensorboard --logdir logs


Launching TensorBoard...

In [96]:
!kill 164724

'kill' is not recognized as an internal or external command,
operable program or batch file.
